## import dataset - LFITW

https://docs.voxel51.com/dataset_zoo/datasets.html#dataset-zoo-activitynet-100

In [6]:
import os
import fiftyone as fo
import fiftyone.zoo as foz

current_dir = os.getcwd()
dataset_dir = os.path.join(current_dir, "../dataset")
os.makedirs(dataset_dir, exist_ok=True)

print(f"Dataset directory created at: {dataset_dir}")

try:
    dataset = foz.load_zoo_dataset(
        "lfw",
        split="test",
        dataset_dir=dataset_dir
    )

    session = fo.launch_app(dataset)

except Exception as e:
    print(f"Error loading dataset: {e}")

Dataset directory created at: /Users/ads/git/toronto-visual-ai-hackathon-2025/notebooks/../dataset
Error loading dataset: HTTPConnectionPool(host='vis-www.cs.umass.edu', port=80): Max retries exceeded with url: /lfw/lfw.tgz (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x153d98990>: Failed to resolve 'vis-www.cs.umass.edu' ([Errno 8] nodename nor servname provided, or not known)"))


In [ ]:
import os
import shutil

source_path = "/Users/ads/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4"
destination_path = "/Users/ads/git/toronto-visual-ai-hackathon-2025/dataset"

if not os.path.exists(source_path):
    print(f"Source path doesn't exist: {source_path}")
    exit()

files = os.listdir(source_path)
print(f"Found {len(files)} files/directories to copy.")

for item in files:
    source_item = os.path.join(source_path, item)
    dest_item = os.path.join(destination_path, item)

    if os.path.isdir(source_item):
        if os.path.exists(dest_item):
            shutil.rmtree(dest_item)
        shutil.copytree(source_item, dest_item)
        print(f"Copied directory: {item}")
    else:
        shutil.copy2(source_item, dest_item)
        print(f"Copied file: {item}")

print(f"\nAll files successfully copied to {destination_path}")

dest_files = os.listdir(destination_path)
print(f"Destination directory now contains {len(dest_files)} files/directories.")

Found 11 files/directories to copy.
Copied file: pairs.csv
Copied file: matchpairsDevTest.csv
Copied file: people.csv
Copied file: lfw_readme.csv
Copied file: peopleDevTest.csv
Copied file: mismatchpairsDevTrain.csv
Copied directory: lfw-deepfunneled
Copied file: matchpairsDevTrain.csv
Copied file: mismatchpairsDevTest.csv
Copied file: lfw_allnames.csv
Copied file: peopleDevTrain.csv

All files successfully copied to /Users/ads/git/toronto-visual-ai-hackathon-2025/dataset
Destination directory now contains 11 files/directories.


## convert to FO

https://docs.voxel51.com/user_guide/dataset_creation/datasets.html#imageclassificationdirectorytree

In [15]:
import os
import fiftyone as fo

current_dir = os.getcwd()
if os.path.basename(current_dir) == "notebooks":
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir

name = "lfw-deepfunneled"
dataset_dir = os.path.join(project_root, "dataset", "lfw-deepfunneled", "lfw-deepfunneled")

print(f"Loading dataset from: {dataset_dir}")

dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    name=name,
)

print(dataset)

print(dataset.head())

# session = fo.launch_app(dataset)

Loading dataset from: /Users/ads/git/toronto-visual-ai-hackathon-2025/dataset/lfw-deepfunneled/lfw-deepfunneled
 100% |█████████████| 13233/13233 [2.3s elapsed, 0s remaining, 5.8K samples/s]      
Name:        lfw-deepfunneled
Media type:  image
Num samples: 13233
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
[<Sample: {
    'id': '67dedb2d40259dc46d042e93',
    'media_type': 'image',
    'filepath': '/Users/ads/git/toronto-visual-ai-hackathon-2025/dataset/lfw-deepfunneled/

In [ ]:
import fiftyone as fo

dataset = fo.load_dataset("lfw-deepfunneled")

session = fo.launch_app(dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v1.4.0

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Discord community 🚀🚀🚀
|  https://community.voxel51.com/
|



## upload to hf

https://docs.voxel51.com/integrations/huggingface.html#huggingface-hub

In [ ]:
import fiftyone as fo
import os
import tempfile
from huggingface_hub import HfApi, login

dataset = fo.load_dataset("lfw-deepfunneled")
print(f"Loaded dataset: {dataset.name}")
print(f"Number of samples: {len(dataset)}")

temp_dir = tempfile.mkdtemp()
print(f"Exporting dataset to temporary directory: {temp_dir}")

dataset.export(
    export_dir=temp_dir,
    dataset_type=fo.types.FiftyOneDataset,
)

try:
    login()

    repo_id = "GangGreenTemperTatum/lfw-deepfunneled"

    api = HfApi()
    api.create_repo(repo_id=repo_id, exist_ok=True)
    print(f"Created repository: {repo_id}")

    print("Starting upload (this may take some time)...")
    api.upload_folder(
        folder_path=temp_dir,
        repo_id=repo_id,
        repo_type="dataset"
    )
    print("Successfully uploaded dataset to Hugging Face Hub!")

except Exception as e:
    print(f"Error pushing to Hugging Face Hub: {e}")
    print("Make sure you're logged in to Hugging Face using `huggingface-cli login`")

# import shutil
# shutil.rmtree(temp_dir)
# print(f"Cleaned up temporary directory: {temp_dir}")

Loaded dataset: lfw-deepfunneled
Number of samples: 13233
Exporting dataset to temporary directory: /var/folders/zd/874qqpwn4p71dft28ltsqn700000gn/T/tmpm68lruz_
Directory '/var/folders/zd/874qqpwn4p71dft28ltsqn700000gn/T/tmpm68lruz_' already exists; export will be merged with existing files
Exporting samples...
 100% |████████████████| 13233/13233 [15.3s elapsed, 0s remaining, 964.0 docs/s]      


It seems you are trying to upload a large folder at once. This might take some time and then fail if the folder is too large. For such cases, it is recommended to upload in smaller batches or to use `HfApi().upload_large_folder(...)`/`huggingface-cli upload-large-folder` instead. For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/upload#upload-a-large-folder.


Created repository: GangGreenTemperTatum/lfw-deepfunneled
Starting upload (this may take some time)...


Askar_Akayev_0001.jpg:   0%|          | 0.00/8.97k [00:00<?, ?B/s]

Astou_Ndiaye-Diatta_0001.jpg:   0%|          | 0.00/9.42k [00:00<?, ?B/s]

...

Buddy_Ryan_0001.jpg:   0%|          | 0.00/9.70k [00:00<?, ?B/s]

Buford_Blount_0001.jpg:   0%|          | 0.00/8.39k [00:00<?, ?B/s]

Bulent_Ecevit_0001.jpg:   0%|          | 0.00/9.66k [00:00<?, ?B/s]

Bulent_Ecevit_0002.jpg:   0%|          | 0.00/7.74k [00:00<?, ?B/s]

Error pushing to Hugging Face Hub: Error while uploading 'data/Bill_Graham_0008.jpg' to the Hub.
Make sure you're logged in to Hugging Face using `huggingface-cli login`
